<a href="https://colab.research.google.com/github/UmardrazArshad/Quarter_3/blob/main/03_tools_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install openai-agents SDK

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00


# Make your Jupyter Notebook capable of running asynchronous functions.

In [2]:
import nest_asyncio
nest_asyncio.apply()

# Run Google Gemini with OPENAI-Agent SDK

In [4]:
import os
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig
from google.colab import userdata


gemini_api_key = userdata.get("GEMINI_API_KEY")
search_api_key = userdata.get("TAVILY_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

In [5]:
import os
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, set_default_openai_client
from agents.run import RunConfig
from google.colab import userdata


gemini_api_key = userdata.get("GEMINI_API_KEY")


external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
set_default_openai_client(external_client)

# Tools
The OpenAI Agents SDK provides a robust framework for integrating various tools into agents, enabling them to perform tasks such as data retrieval, web searches, and code execution. Here's an overview of the key points regarding tool integration:

**Types of Tools:**

1. **Hosted Tools:** These are pre-built tools running on OpenAI's servers, accessible via the [`OpenAIResponsesModel`]. Examples include:
   - **WebSearchTool:** Enables agents to perform web searches.
   - **FileSearchTool:** Allows retrieval of information from OpenAI Vector Stores.
   - **ComputerTool:** Facilitates automation of computer-based tasks.

2. **Function Calling:** This feature allows agents to utilize any Python function as a tool, enhancing their versatility.

3. **Agents as Tools:** Agents can employ other agents as tools, enabling hierarchical task management without transferring control.

**Implementing Tools:**

- **Function Tools:** By decorating Python functions with `@function_tool`, they can be seamlessly integrated as tools for agents.

**Tool Execution Flow:**

- During an agent's operation, if a tool call is identified in the response, the SDK processes the tool call, appends the tool's response to the message history, and continues the loop until a final output is produced.

**Error Handling:**

- The SDK offers mechanisms to handle errors gracefully, allowing agents to recover from tool-related issues and continue their tasks effectively.

For a comprehensive understanding and implementation details, refer to the [tools documentation](https://github.com/openai/openai-agents-python/blob/main/docs/tools.md).

# Create tools

In [6]:
from agents.tool import function_tool

In [ ]:
# @function_tool("get_weather")
# def get_weather(location: str, unit: str = "C") -> str:
#   """
#   Fetch the weather for a given location, returning a short description.
#   """
#   # Example logic
#   return f"The weather in {location} is 22 degrees {unit}."

In [ ]:
# @function_tool("piaic_student_finder")
# def student_finder(student_roll: int) -> str:
#   """
#   find the PIAIC student based on the roll number
#   """
#   data = {1: "Qasim",
#           2: "Sir Zia",
#           3: "Daniyal"}

#   return data.get(student_roll, "Not Found")

In [7]:
@function_tool
def search_online(query: str) -> str:
  """
  Search the web for the given query.
  """
  from tavily import TavilyClient

  tavily_client = TavilyClient(api_key="search_api_key")
  response = tavily_client.search("Who is Leo Messi?")

  print(response)

# Connect tools with Agent

In [8]:
import asyncio

from agents import Agent, Runner


async def main():
    agent = Agent(
        name="Assistant",
        instructions="you can search online or give simply answers.",
        tools=[search_online], # add tools here
        model=model
    )

    result = Runner.run_sync(agent, "who is imran khan")
    print(result.final_output)



if __name__ == "__main__":
    asyncio.run(main())

Imran Khan is a Pakistani politician and former cricketer who served as the 22nd Prime Minister of Pakistan from August 2018 until April 2022. Before entering politics, he was a professional cricketer and captained the Pakistan national cricket team to victory in the 1992 Cricket World Cup. He founded the political party Pakistan Tehreek-e-Insaf (PTI) in 1996 and served as its chairman.

